In [1]:
!pip install duckdb
import duckdb
from datetime import datetime
con = duckdb.connect('/tmp/DuckDB Database')
c = con.cursor()

     |████████████████████████████████| 1.3MB 4.9MB/s 
  Created wheel for duckdb: filename=duckdb-0.1.1-cp36-cp36m-linux_x86_64.whl size=1947905 sha256=11a1860bf6cd55438150ac958c418646da63f2a731ed975a0c3b1c8ebaf27aa9
  Stored in directory: /root/.cache/pip/wheels/31/2e/81/8061e62cf80a0ea73a4657d5807c46a93105440af5921e828c
Successfully built duckdb


In [2]:
!wget -O Robust04.rar https://dl.dropboxusercontent.com/s/yyug8pi399qi2s9/Robust04%2Bpos%2Btf.rar?dl=0
!unrar e -o+ Robust04.rar

--2019-11-14 09:06:49--  https://dl.dropboxusercontent.com/s/yyug8pi399qi2s9/Robust04%2Bpos%2Btf.rar?dl=0
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.1.6, 2620:100:6016:6::a27d:106
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.1.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 459824653 (439M) [application/rar]
Saving to: ‘Robust04.rar’

Robust04.rar        100%[===================>] 438.52M  38.9MB/s    in 12s     

2019-11-14 09:07:02 (37.3 MB/s) - ‘Robust04.rar’ saved [459824653/459824653]


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from Robust04.rar

Extracting  dict.csv                                                       0%  OK 
Extracting  docs.csv                                                       0%  1%  OK 
Extracting  terms.csv                                                      1%  2%  3%  4%  5%  6%

In [3]:
c.execute("CREATE TABLE dict(termid INTEGER "
                           ",term   VARCHAR "
                           ",df     INTEGER)")
c.execute("CREATE TABLE terms(termid INTEGER "
                            ",docid  INTEGER "
                            ",pos    INTEGER "
                            ",tf     INTEGER)")
c.execute("CREATE TABLE docs(name  VARCHAR "
                           ",docid INTEGER "
                           ",len   INTEGER "
                           ",temp  INTEGER)")   

In [4]:
c.execute("COPY dict "
          "FROM 'dict.csv' "
          "WITH DELIMITER '|'")
c.execute("COPY terms "
          "FROM 'terms.csv' "
          "WITH DELIMITER '|'")
c.execute("COPY docs "
          "FROM 'docs.csv' "
          "WITH DELIMITER '|'")

In [0]:
def make_query(c, *args: str):
    """Function to make query table in database.
    
    Args:
        c: cursor of DuckDB
        args: concatenation of strings to be made into a query
        
    Returns [str]:
        Number of terms in the query as string.
    """
    query = "('" + args[0] + "')"
    for arg in args[1:]:
        query += ", ('" + arg + "')"
    
    c.execute("DROP TABLE IF EXISTS query")
    c.execute("CREATE TABLE query(term VARCHAR)")
    c.execute("INSERT INTO query VALUES " + query)     
        
    return str(len(args))

In [25]:
len_query = make_query(c, 'wizard', 'hat')

cur_time = datetime.now()

qterms = c.execute("WITH qtermids "
                        "AS (SELECT dict.termid "
                                  ",dict.df "
                            "FROM dict "
                            "JOIN query "
                            "ON dict.term = query.term "
                            ") "
                   "SELECT terms.termid "
                         ",terms.docid "
                         ",terms.pos "
                         ",terms.tf "
                         ",qtermids.df "
                   "FROM terms "
                   "JOIN qtermids "
                   "ON terms.termid = qtermids.termid")

time_delta = datetime.now() - cur_time
print('Query time: {}'.format(time_delta))

qterms.fetchdf()

Query time: 0:00:00.849248


,termid,docid,pos,tf,df
0,434301,44,589,1,1718
1,434301,152,497,1,1718
2,434301,281,92,1,1718
3,434301,339,1024,1,1718
4,434301,396,696,1,1718
...,...,...,...,...,...
2590,694497,323249,53,2,269
2591,694497,323249,186,2,269
2592,694497,324788,1341,1,269
2593,694497,326675,155,1,269


In [26]:
len_query = make_query(c, 'new', 'york')

N = c.execute("SELECT COUNT(*) "
              "FROM docs")
N = str(N.fetchdf().iloc[0, 0])
avgdl = c.execute("SELECT AVG(docs.len) "
                  "FROM docs")
avgdl = str(avgdl.fetchdf().iloc[0, 0])

k1 = '1.2'
b = '0.75'

k = '20'

cur_time = datetime.now()

t = c.execute("WITH qtermids "
                   "AS (SELECT dict.termid "
                             ",dict.df "
                       "FROM dict "
                       "JOIN query "
                       "ON dict.term = query.term "
                       ") "
              ", qterms "
                   "AS (SELECT terms.termid "
                             ",terms.docid "
                             ",terms.pos "
                             ",terms.tf "
                             ",qtermids.df "
                       "FROM terms "
                       "JOIN qtermids "
                       "ON terms.termid = qtermids.termid"
                       ") "
              ", condocs "
                   "AS (SELECT qterms.docid "
                       "FROM qterms "
                       "GROUP BY qterms.docid "
                       "HAVING COUNT(DISTINCT qterms.termid) = " + len_query +
                       ") "
              ", okapisubscores "
                   "AS (SELECT qterms.docid "
                             ",( LOG((" + N + "-df+0.5)/(df+0.5))*tf*(" + k1 + "+1)"
                                 "/"
                                "(tf+" + k1 + "*(1-" + b + "+" + b + "*len/" + avgdl + "))"
                               ") AS subscore "
                       "FROM qterms "
                       "JOIN condocs "
                       "ON qterms.docid = condocs.docid "
                       "JOIN docs "
                       "ON qterms.docid = docs.docid "
                       ") "
              ", topkdocs "
                   "AS (SELECT okapisubscores.docid "
                       "FROM okapisubscores "
                       "GROUP BY okapisubscores.docid "
                       "ORDER BY SUM(okapisubscores.subscore) DESC "
                       "LIMIT " + k +
                       ") "
              ", pairs "
                   "AS (SELECT qterms1.termid AS termid1 "
                             ",qterms2.termid AS termid2 "
                             ",qterms1.docid "
                             ",1.0/(qterms1.pos - qterms2.pos) AS tpi "
                             ",( CASE WHEN qterms1.df > qterms2.df THEN qterms1.df "
                                     "ELSE qterms2.df "
                                "END"
                               ") AS maxdf "
                       "FROM qterms AS qterms1 "
                       "JOIN qterms AS qterms2 "
                       "ON qterms1.docid = qterms2.docid AND "
                          "NOT qterms1.termid = qterms2.termid AND "
                          "qterms1.pos - qterms2.pos BETWEEN 1 AND 5 "
                      #  "JOIN topkdocs "
                      #  "ON qterms1.docid = topkdocs.docid"
                       ") "
              ",tpisums "
                   "AS (SELECT pairs.termid1 "
                             ",pairs.termid2 "
                             ",pairs.docid "
                             ",SUM(pairs.tpi) AS tpisum "
                       "FROM pairs "              
                       "GROUP BY pairs.termid1 "
                               ",pairs.termid2 "
                               ",pairs.docid"
                       ") "
              ", subscores "
                   "AS (SELECT pairs.docid "
                             ",( LOG((" + N + "-maxdf+0.5)/(maxdf+0.5))*tpisum*(" + k1 + "+1)"
                                 "/"
                                "(tpisum+" + k1 + "*(1-" + b + "+" + b + "*len/" + avgdl + "))"
                               ") AS subscore "
                       "FROM pairs "
                       "JOIN tpisums "
                       "ON pairs.termid1 = tpisums.termid1 AND "
                          "pairs.termid2 = tpisums.termid2 AND "
                          "pairs.docid = tpisums.docid "
                       "JOIN docs "
                       "ON pairs.docid = docs.docid"
                       ") "
              ", scores "
                   "AS (SELECT subscores.docid "
                             ",SUM(subscores.subscore) AS score "
                       "FROM subscores "
                       "GROUP BY subscores.docid"
                       ") "
              "SELECT scores.docid "
                    ",scores.score "
              "FROM scores "
              # "JOIN topkdocs "
              # "ON scores.docid = topkdocs.docid "
              # "WHERE scores.docid BETWEEN 50961 AND 84012 "
              "ORDER BY scores.score DESC")

time_delta = datetime.now() - cur_time
print('Query time: {}'.format(time_delta))

t.fetchdf()

Query time: 0:00:03.657094


,docid,score
0,50961,55.958108
1,19577,16.473072
2,181752,13.586938
3,139362,13.327779
4,238507,12.143474
...,...,...
3803,154700,0.043961
3804,145910,0.038490
3805,121550,0.025823
3806,251365,0.024502
